In [243]:
import pandas as pd
import numpy as np
import os
from functools import reduce
from tabulate import tabulate

In [278]:
total = []
std = pd.read_csv('../Data/Intermediate/CDCSTDoverall2000_2020_processed.csv', index_col = 0)
covid = pd.read_csv('../Data/Intermediate/COVIDcounty_cases_processed_2020.csv', index_col = 0)
dp05 = pd.read_csv('../Data/Intermediate/ACSDP05_merge.csv', index_col = 0)
dp03 = pd.read_csv('../Data/Intermediate/ACSDP03_merge.csv', index_col = 0)
saipe = pd.read_csv('../Data/Intermediate/saipe0210.csv', index_col = 0)
total.extend([std,covid,dp03,dp05, saipe ])
#print(total)

In [245]:
dp05['total_pop'] = dp05['total_pop'].astype('float64')

In [246]:
dp03[dp03['fips']==1001]

,unemployment_rate,employment_pop_ratio,pct_ind_agriculture_forestry_fishing_and_hunting_and_mining,pct_ind_construction,pct_ind_manufacturing,pct_ind_wholesale_trade,pct_ind_retail_trade,pct_ind_transportation_and_warehousing_and_utilities,pct_ind_information,pct_ind_finance_and_insurance_and_real_estate_and_rental_and_leasing,pct_ind_professional_scientific_and_management_and_administrative_and_waste_management_services,pct_ind_educational_services_and_health_care_and_social_assistance,pct_ind_arts_entertainment_and_recreation_and_accommodation_and_food_services,pct_ind_other_services_except_public_administration,pct_ind_public_administration,health_insurance_rate,poverty_rate,year,fips,per_capita_income_2022
0,4.0,60.4,0.9,7.7,14.1,3.1,12.7,4.4,1.1,6.5,7.0,17.4,8.7,4.6,11.7,NaN,10.6,2010,1001,33412.48
1,4.8,58.7,1.2,7.0,14.0,3.6,12.5,5.3,1.0,6.2,7.1,18.3,8.2,4.4,11.3,NaN,10.9,2011,1001,32795.85
2,5.4,57.7,1.4,5.6,13.6,3.0,13.5,5.7,1.1,6.2,7.4,18.8,7.8,4.5,11.4,90.4,11.6,2012,1001,32576.37
3,5.9,56.7,1.4,5.8,12.9,3.0,13.5,5.9,1.1,6.0,7.8,17.7,7.6,5.9,11.4,88.7,12.1,2013,1001,31205.17
4,5.3,56.8,1.6,5.6,13.2,3.2,12.8,5.2,1.3,6.0,8.2,17.9,7.6,6.2,11.3,89.3,12.8,2014,1001,30558.56
5,7.6,56.0,1.4,5.4,14.1,3.4,11.2,5.9,1.2,5.2,7.7,18.8,7.9,6.4,11.2,89.9,12.9,2015,1001,30967.76
6,5.6,56.8,1.2,4.8,14.0,3.1,10.4,6.6,1.8,5.5,8.3,18.2,7.6,6.6,11.9,91.1,12.3,2016,1001,32186.64
7,5.2,56.1,0.9,5.1,13.3,3.1,9.8,7.4,1.7,5.6,8.1,17.6,8.9,7.2,11.2,91.2,13.7,2017,1001,33667.04
8,4.2,55.6,0.9,6.0,12.9,2.7,8.9,7.0,1.6,6.3,8.9,20.1,8.1,6.2,10.3,92.9,15.4,2018,1001,34658.96
9,3.7,55.8,0.9,6.1,13.0,2.5,9.9,6.8,1.4,6.0,9.6,20.2,8.8,5.4,9.4,92.9,15.2,2019,1001,34590.04


In [284]:
for index, i in enumerate(total):
    i['fips'] = i['fips'].astype(str).str.zfill(5)
    print(i[['year', 'fips']].dtypes)
    if 'year' in i.columns:
        if 'area' in i.columns:
            i.drop(columns = 'area', inplace = True)
            print(index, ': ', 'Done')
        if 'county_name' in i.columns:
            i.drop(columns = 'county_name' , inplace = True)
            print(index, ': ', 'done')
        if 'state_fips' in i.columns:
            i.drop(columns = 'state_fips', inplace = True)
            print(index, ': ', 'DONE')

year     int64
fips    object
dtype: object
year     int64
fips    object
dtype: object
1 :  done
year     int64
fips    object
dtype: object
year     int64
fips    object
dtype: object
year     int64
fips    object
dtype: object


In [265]:
cov = total[0].columns.intersection(total[1].columns)
df = pd.merge(total[0], total[1], on = cov.to_list(), how = 'outer')
df = df[(df['year'] >= 2002) & (df['year'] <= 2020)]
df = df.sort_values(by = ['fips','year']).reset_index(drop = True)
print(cov)
for i in cov:
    print(total[0][i].dtypes, total[1][i].dtypes)
print(df['year'].value_counts())
print(df.shape)
df.head(20)

Index(['year', 'fips'], dtype='object')
int64 int64
object object
2020    3222
2012    3216
2019    3216
2018    3216
2017    3216
2016    3216
2015    3216
2014    3216
2013    3216
2002    3216
2003    3216
2010    3216
2009    3216
2008    3216
2007    3216
2006    3216
2005    3216
2004    3216
2011    3216
Name: year, dtype: int64
(61110, 131)


,year,fips,cases_chlamydia,cases_early_non-primary_non-secondary_syphilis,cases_gonorrhea,cases_primary_and_secondary_syphilis,rate_per_100000_chlamydia,rate_per_100000_early_non-primary_non-secondary_syphilis,rate_per_100000_gonorrhea,rate_per_100000_primary_and_secondary_syphilis,...,rate_per_100000_male_gonorrhea_native_hawaiian_other_pacific_islander,rate_per_100000_male_gonorrhea_white,rate_per_100000_male_primary_and_secondary_syphilis_american_indian_alaska_native,rate_per_100000_male_primary_and_secondary_syphilis_asian,rate_per_100000_male_primary_and_secondary_syphilis_black_african_american,rate_per_100000_male_primary_and_secondary_syphilis_hispanic_latino,rate_per_100000_male_primary_and_secondary_syphilis_multiracial,rate_per_100000_male_primary_and_secondary_syphilis_native_hawaiian_other_pacific_islander,rate_per_100000_male_primary_and_secondary_syphilis_white,covid_cases
0,2002,01001,72.0,4.0,54.0,3.0,159.3,8.8,119.5,6.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003,01001,95.0,0.0,71.0,2.0,207.6,0.0,155.2,4.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2004,01001,86.0,2.0,68.0,0.0,183.2,4.3,144.8,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2005,01001,97.0,0.0,64.0,0.0,202.6,0.0,133.7,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2006,01001,183.0,0.0,81.0,0.0,372.6,0.0,164.9,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2007,01001,184.0,0.0,104.0,1.0,369.2,0.0,208.7,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2008,01001,155.0,4.0,66.0,5.0,307.9,7.9,131.1,9.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2009,01001,214.0,0.0,53.0,3.0,421.1,0.0,104.3,5.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2010,01001,198.0,3.0,30.0,2.0,361.5,5.5,54.8,3.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2011,01001,247.0,0.0,58.0,1.0,447.2,0.0,105.0,1.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [292]:
for i in total[2:5]:
    #print(list(i.dtypes))
    cov = df.columns.intersection(i.columns)
    df = pd.merge(df, i, on = cov.to_list(), how = 'outer')
    df = df[(df['year'] >= 2002) & (df['year'] <= 2020)]
    df = df.sort_values(by = ['fips','year']).reset_index(drop = True)
    print(df['year'].value_counts())
    print(cov)
    print(df.head())
    # for j in cov:
    #     print(df[j].dtypes, i[j].dtypes)
    #print(df.shape)

2011    3225
2010    3225
2013    3225
2012    3225
2014    3224
2020    3224
2015    3222
2016    3222
2017    3222
2018    3222
2019    3222
2008    3216
2009    3216
2007    3216
2003    3216
2006    3216
2005    3216
2004    3216
2002    3216
Name: year, dtype: int64
Index(['year', 'fips'], dtype='object')
   year   fips  cases_chlamydia  \
0  2002  01001             72.0   
1  2003  01001             95.0   
2  2004  01001             86.0   
3  2005  01001             97.0   
4  2006  01001            183.0   

   cases_early_non-primary_non-secondary_syphilis  cases_gonorrhea  \
0                                             4.0             54.0   
1                                             0.0             71.0   
2                                             2.0             68.0   
3                                             0.0             64.0   
4                                             0.0             81.0   

   cases_primary_and_secondary_syphilis  rate_per_100000

In [253]:
for i in df.index:
    if df.loc[i, 'year'] <=2009 :
        df.loc[i, 'employment_pop_ratio'] = df.loc[i,'employed_persons']/df.loc[i,'total_pop'] * 100
df['employment_pop_ratio'].head(20)

0     60.4
1     58.7
2     57.7
3     56.7
4     56.8
5     56.0
6     56.8
7     56.1
8     55.6
9     55.8
10    55.4
11    56.2
12    56.0
13    55.5
14    54.8
15    54.7
16    55.0
17    54.7
18    54.7
19    55.7
Name: employment_pop_ratio, dtype: float64

In [254]:
df.drop(columns = 'employed_persons', inplace = True)


In [255]:
counts = df['fips'].value_counts()
ys = df[df['fips'].isin(counts[counts == 19].index)]

ys.head(10)

,unemployment_rate,employment_pop_ratio,pct_ind_agriculture_forestry_fishing_and_hunting_and_mining,pct_ind_construction,pct_ind_manufacturing,pct_ind_wholesale_trade,pct_ind_retail_trade,pct_ind_transportation_and_warehousing_and_utilities,pct_ind_information,pct_ind_finance_and_insurance_and_real_estate_and_rental_and_leasing,...,rate_per_100000_male_gonorrhea_native_hawaiian_other_pacific_islander,rate_per_100000_male_gonorrhea_white,rate_per_100000_male_primary_and_secondary_syphilis_american_indian_alaska_native,rate_per_100000_male_primary_and_secondary_syphilis_asian,rate_per_100000_male_primary_and_secondary_syphilis_black_african_american,rate_per_100000_male_primary_and_secondary_syphilis_hispanic_latino,rate_per_100000_male_primary_and_secondary_syphilis_multiracial,rate_per_100000_male_primary_and_secondary_syphilis_native_hawaiian_other_pacific_islander,rate_per_100000_male_primary_and_secondary_syphilis_white,covid_cases
37148,12.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37149,11.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37150,13.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37151,13.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37152,12.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37153,12.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37154,12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37155,15.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [256]:
ys.to_csv('../Data/Final/STI0220_cov0220.csv')